In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install --upgrade tensorflow tensorflow-addons pandas matplotlib

In [ ]:
!rm -rf /content/drive/MyDrive/PokemonGenFinal
!unzip /content/drive/MyDrive/PokemonGenFinal.zip -d /content/drive/MyDrive/

In [ ]:
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split

# Load data from PokemonDataset.csv
csv_file_path = "/content/drive/MyDrive/PokemonGenFinal/pokemon_images.csv"  # Ensure the file path is correct
data_df = pd.read_csv(csv_file_path)

# Add prefix to the image_path column
base_path = "/content/drive/MyDrive/PokemonGenFinal/"
data_df['image_path'] = base_path + data_df['image_path'].astype(str)

# Encode labels (name column)
class_labels = data_df['name'].unique()  # Get unique names as class labels
label_to_index = {label: idx for idx, label in enumerate(class_labels)}  # Create a mapping
data_df['label'] = data_df['name'].map(label_to_index)  # Map names to numerical labels

# Filter out classes with only one instance
data_df = data_df[data_df['label'].map(data_df['label'].value_counts()) > 1]

# Train-test split
train_df, val_df = train_test_split(data_df, test_size=0.2, stratify=data_df['label'])

# Preprocessing function
IMG_SIZE = 224  # MobileNetV3Large input size is 224x224
def preprocess_image(image_path, label):
    image = tf.io.read_file(image_path)  # Read the image file
    image = tf.image.decode_jpeg(image, channels=3)  # Decode the image
    image = tf.image.resize(image, [IMG_SIZE, IMG_SIZE])  # Resize to IMG_SIZE
    image = tf.keras.applications.mobilenet_v3.preprocess_input(image)  # Preprocess for MobileNetV3
    return image, label

# Create TensorFlow datasets
def create_dataset(df):
    file_paths = df['image_path'].values  # Extract file paths
    labels = df['label'].values  # Extract labels
    dataset = tf.data.Dataset.from_tensor_slices((file_paths, labels))  # Create a TensorFlow dataset
    dataset = dataset.map(preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)  # Preprocess images
    dataset = dataset.shuffle(buffer_size=1000).batch(32).prefetch(buffer_size=tf.data.AUTOTUNE)  # Optimize dataset
    return dataset

# Create train and validation datasets
train_dataset = create_dataset(train_df)
val_dataset = create_dataset(val_df)

# Print a summary
print(f"Number of training samples: {len(train_df)}")
print(f"Number of validation samples: {len(val_df)}")
print(f"Class to label mapping: {label_to_index}")

In [ ]:
from tensorflow.keras.models import load_model

# Load the trained model
fine_tune_model = load_model("/content/drive/MyDrive/pre_train_mobile.keras")

for i, layer in enumerate(fine_tune_model.layers):
    print(f"Layer {i}: {layer.name}, Trainable: {layer.trainable}")

In [ ]:
# Evaluate the model on the validation dataset
results = fine_tune_model.evaluate(val_dataset)

# Print the evaluation results
print("\nValidation Results:")
for metric_name, metric_value in zip(fine_tune_model.metrics_names, results):
    print(f"{metric_name}: {metric_value}")

In [ ]:
# Unfreeze the base model (MobileNetV3Large)
for layer in fine_tune_model.layers:
    layer.trainable = True

for i, layer in enumerate(fine_tune_model.layers):
    print(f"Layer {i}: {layer.name}, Trainable: {layer.trainable}")

In [ ]:
# Fine-Tuning
fine_tune_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss='sparse_categorical_crossentropy',
              metrics=['sparse_categorical_accuracy'])

history_fine_tune = fine_tune_model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=25
)